In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [6]:
#read in reference file
ref_5k = pd.read_csv('ref_75.txt', delimiter=' ', header = None)


In [44]:
#convert reference file into 3 part prob dist file
#for each SNP, each person, who is 100% AA, AB, or BB, which is written as 00, 01, or 11, gets 
#turned into one number for AA, one number for AB, one number for BB (so 1, 0,0 as AA, 0, 1, 0 as
#AB, and 0,0,1 for BB)

new_dist = np.zeros(())

SNPs = int(ref_5k.shape[0])
people = int((ref_5k.shape[1]-5) / 2.0)

new_dist = np.zeros((SNPs, people * 3))

#need to come back to this...
for SNP in range(SNPs):
    for person in range(people):
        if ref_5k.loc[SNP, 5 + person*2] == 0 & 
    new_dist[SNP, 0] = ref_5k[SNP, 0]
#for SNP in range(SNPs):




ValueError: could not convert string to float: 'A'